In [63]:
from collections import defaultdict
import numpy as np
import json
import facebook
import unicodedata
import requests
import pandas as pd
import dateutil.parser as dateparser
import calendar
from datetime import datetime, timedelta

In [174]:
#contains access token
access = 'CAACEdEose0cBAP0wZAyZBE1mEenNHD9T7jurZAczzMYIFjwbv9wYnZCGimUW4CGkKMXMyXm9X8bs12c0q9kSujWe50AsMeU0vu7sOP1Wb0lZAEUP0JyhfXgCXqU4S2DeuGm1rQxXWboZByKZBbLuLPLq2vPgA3ZCAT9Hn5ImM7KiZCuNZChjIjx8C6YVbfRlZA3Nwc35fw3nP2sqAZDZD'

In [175]:
graph = facebook.GraphAPI(access)

In [6]:
#appends the post IDs to a list
post_ids = []
def get_post_ids(post):
    post_ids.append(post['id'])

In [7]:
#obtain the most recent 25 facebook posts
posts = graph.get_connections('me', 'posts', **{'fields': 'id'})

In [8]:
#run a while loop 20 times to obtain the the most recent 500 posts
i = 0
while i < 20:
    [get_post_ids(post) for post in posts['data']]
    posts = requests.get(posts['paging']['next']).json()
    i += 1

In [9]:
print len(post_ids)

500


In [10]:
#obtain the date of the 500th recent post
print posts['data'][-1]

{u'created_time': u'2015-11-25T05:30:04+0000', u'id': u'127466294080089_508889732604408'}


In [11]:
#obtain a list containing the names and IDs of all people who liked the most recent 500 posts
likers = []
for ID in post_ids:
    likes = graph.get_connections(ID, 'likes')
    for dic in likes['data']:
        likers.append((dic['id'], dic['name']))

In [12]:
print len(likers)

1561


In [13]:
#make a dictionary of the list of likers
d = defaultdict(int)
for i in likers:
    d[i]+=1

In [14]:
#remove people who have liked less than 4 posts out of the most recent 500 posts
new_d = defaultdict(int)
for item in d:
    if d[item]>3:
        new_d[item] = d[item]

In [15]:
#sort likers in descending order
sorted_list = sorted(new_d.items(), key = lambda x:x[1])

In [16]:
#make lists, to be used as columns for dataframe later
names = []
frequencies = []
IDs = []
for item in sorted_list:
    id_name, frequency = item
    ID, name = id_name
    names.append(name)
    IDs.append(ID)
    frequencies.append(frequency)

In [17]:
#make empty dataframe
df = pd.DataFrame()

In [18]:
#add columns to dataframe
df['Name'] = names
df['ID'] = IDs
df['Number of Posts Liked'] = frequencies
df['Link to Profile'] = 'https://www.facebook.com/'+df['ID']

In [19]:
#re-arrange columns
df = df[['Name', 'ID', 'Link to Profile', 'Number of Posts Liked']]

In [20]:
#export columns as csv
df[['Name', 'Link to Profile', 'Number of Posts Liked']].to_csv('/home/ketan/Desktop/Likers_List.csv', encoding = 'utf-8')

In [21]:
#get number of page likes
def number_of_likes(ID):
    page = graph.get_object(ID)
    try:
        result = page['likes']
    except KeyError:
        result = '-'
    return result

In [22]:
#determines if a user likes a page or not
def user_likes_page(user_id, page_id):
    url = 'https://graph.facebook.com/%d/likes/%d/' % (user_id, page_id)
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    if result['data']:
        return True
    else:
        return False

In [23]:
#list containing number of Page Likes
page_likes_list = []
for ID in df['ID']:
    page_likes_list.append(number_of_likes(ID))

In [24]:
likes_safecity = []
for ID in df['ID']:
    if user_likes_page(int(ID), 127466294080089):
        likes_safecity.append('Yes')
    else:
        likes_safecity.append('No')

In [37]:
for i in names:
    if i == 'Safecity':
        safecity_index = names.index(i)
likes_safecity[safecity_index] = '-'

In [38]:
df['Number of Page Likes'] = page_likes_list

In [42]:
df['Likes Safecity'] = likes_safecity

In [45]:
df = df.sort_values(['Number of Posts Liked'], ascending = False)

In [46]:
df[['Name', 'Link to Profile', 'Number of Posts Liked', 'Number of Page Likes', 'Likes Safecity']].to_csv('/home/ketan/Desktop/List of Likers.csv', encoding = 'utf-8')

## Assignment 2: Post Analysis

In [48]:
#adding 500 more post IDs to the list post_ids
i = 0
while i < 40:
    if i >= 20:
        [get_post_ids(post) for post in posts['data']]
        posts = requests.get(posts['paging']['next']).json()
        i += 1
    else:
        i += 1 

In [68]:
len(post_ids)

1000

In [49]:
#get number of likes on a post
def get_post_likes(post_id):
    count_likes = 0
    url = 'https://graph.facebook.com/%s/likes/' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    final_result = []
    result = json.loads(r.text)
    while True:
        try:
            [final_result.append(dic) for dic in result['data']]
            result = requests.get(result['paging']['next']).json()
        except KeyError:
            break
    return len(final_result)

In [50]:
#get number of people who like both the post, and the page
def get_common_likes_generalized(post_id, page_id): #post_id is a string, page_id is an int
    count_likes = 0
    url = 'https://graph.facebook.com/%s/likes/' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for like in result['data']:
        if user_likes_page(int(like['id']), page_id):
            count_likes+=1
    return count_likes

In [51]:
#get number of people who like both the post, and safecity's facebook page
def get_common_likes_safecity(post_id):
    count_likes = 0
    url = 'https://graph.facebook.com/%s/likes/' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for like in result['data']:
        if user_likes_page(int(like['id']), 127466294080089):
            count_likes+=1
    return count_likes

In [52]:
#get the created time for a post
def get_post_date(post_id):
    url = 'https://graph.facebook.com/%s' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    return result['created_time']

In [53]:
#take facebook created_time, and convert it to Python datetime format, in the local time zone
def facebook_to_local(time):
    x = dateparser.parse(time)
    timestamp = calendar.timegm(x.timetuple())
    local_dt = datetime.fromtimestamp(timestamp)
    assert x.resolution >= timedelta(microseconds=1)
    return local_dt.replace(microsecond=x.microsecond)

#### NOT GETTING THE CORRECT NUMBER OF SHARES FOR SOME POSTS DUE TO A BUG IN FACEBOOK

In [54]:
#get number of shares of a post
def num_shares(post):
    url = 'https://graph.facebook.com/%s' % post
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        x = result['shares']['count']
    except KeyError:
        x = 0
    return x

In [55]:
#get status type
def get_status_type(post_id):
    url = 'https://graph.facebook.com/%s' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    return result['status_type']

In [56]:
#get the type of story for a post
def post_story(post):
    url = 'https://graph.facebook.com/%s/' %post
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        ans = result['story']
    except KeyError:
        ans = '-'
    return ans

In [57]:
#get the total number of people the Page post was served to
def get_post_reach(post_id):
    url = 'https://graph.facebook.com/%s/insights' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for dic in result['data']:
        if dic['name'] == 'post_impressions_unique':
            return dic['values'][0]['value']

In [58]:
#get the number of impressions of the Page post
def get_post_impressions(post_id):
    url = 'https://graph.facebook.com/%s/insights' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for dic in result['data']:
        if dic['name'] == 'post_impressions':
            return dic['values'][0]['value']

In [59]:
#get the number of people who saw the Page post in a story from a friend
def get_post_viral_reach(post_id):
    url = 'https://graph.facebook.com/%s/insights' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for dic in result['data']:
        if dic['name'] == 'post_impressions_viral_unique':
            return dic['values'][0]['value']

In [60]:
#get the number of impressions of the Page post in a story generated by a friend
def get_post_viral_impressions(post_id):
    url = 'https://graph.facebook.com/%s/insights' % post_id
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    for dic in result['data']:
        if dic['name'] == 'post_impressions_viral':
            return dic['values'][0]['value']

In [136]:
#get post message
def get_post_message(ID):
    url = 'https://graph.facebook.com/%s' % ID
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        x = result['message']
    except KeyError:
        x = '-'
    return x

In [147]:
#determines if a Facebook post had been posted via twitter
def via_twitter(ID):
    url = 'https://graph.facebook.com/%s' % ID
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        if result['application']['namespace'] == 'twitter':
            x = 'Yes'
        else:
            x = 'No'
    except KeyError:
        x = 'No'
    return x

In [165]:
#get the text message of a post
def get_message(ID):
    url = 'https://graph.facebook.com/%s' % ID
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        x = result['message']
    except KeyError:
        x = '-'
    return ''.join(x.splitlines())

In [169]:
#get the title of the Shared Link
def post_name(post):
    url = 'https://graph.facebook.com/%s' % post
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        x = result['name']
    except KeyError:
        x = '-'
    return x

In [61]:
df2 = pd.DataFrame()

In [66]:
df2['Post ID'] = post_ids
df2['Link to Post'] = 'https://facebook.com/'+df2['Post ID']
df2['Total Number of Post Likes'] = df2['Post ID'].apply(get_post_likes)

In [67]:
df2['Number of Likes by Followers'] = df2['Post ID'].apply(get_common_likes_safecity)

In [70]:
df2['Number of Likes by Non-Followers'] = df2['Total Number of Post Likes'] - df2['Number of Likes by Followers']

In [71]:
dt = []
for ID in df2['Post ID']:
    dt.append(facebook_to_local(get_post_date(ID)))

In [72]:
dates = []
times = []
for i in dt:
    dates.append(i.date())
    times.append(str(i.time())[:5])

In [73]:
df2['Date Created'] = dates
df2['Time Created'] = times

#### NOT GETTING THE CORRECT NUMBER OF SHARES FOR SOME POSTS

In [106]:
df2['Number of Shares'] = df2['Post ID'].apply(num_shares)  #not accurate

In [76]:
df2['Post Type'] = df2['Post ID'].apply(get_status_type)

In [80]:
df2['Post Story'] = df2['Post ID'].apply(post_story)

In [81]:
df2['Post Reach'] = df2['Post ID'].apply(get_post_reach)

In [85]:
df2['Post Impressions'] = df2['Post ID'].apply(get_post_impressions)

In [86]:
df2['Post Viral Reach'] = df2['Post ID'].apply(get_post_viral_reach)

In [87]:
df2['Post Viral Impressions'] = df2['Post ID'].apply(get_post_viral_impressions)

In [140]:
df2['Post Message'] = df2['Post ID'].apply(get_post_message)

In [155]:
df2['Posted via Twitter'] = df2['Post ID'].apply(via_twitter)

/home/ketan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [167]:
df2['Post Message'] = df2['Post ID'].apply(get_message)

In [176]:
df2['Title'] = df2['Post ID'].apply(post_name)

In [89]:
post_type_dic = defaultdict(int)
for item in df2['Post Type']:
    post_type_dic[str(item)] += 1
post_type_dic

defaultdict(int,
            {'added_photos': 36,
             'added_video': 1,
             'created_event': 4,
             'mobile_status_update': 259,
             'shared_story': 700})

In [108]:
count = 0
for n, m in zip(df2['Number of Likes by Followers'], df2['Number of Likes by Non-Followers']):
    if m>=n:
        count += 1
count/1000. * 100

78.10000000000001

In [177]:
df2.to_csv('/home/ketan/Desktop/Based on Posts.csv', encoding = 'utf-8')

In [95]:
DF = pd.read_csv('/home/ketan/Desktop/1000 Posts.csv')

In [161]:
twitter_count = 0
twitter_ids = []
for ID in df2['Post ID']:
    index = df2['Post ID'].tolist().index(ID)
    url = 'https://graph.facebook.com/%s' % ID
    parameters = {'access_token': access}
    r = requests.get(url, params = parameters)
    result = json.loads(r.text)
    try:
        x = result['application']['namespace'] 
    except KeyError:
        x = 'no'
    if x == 'twitter':
        twitter_ids.append(df2['Post ID'].tolist()[index])